In [1]:
import numpy as np
from qiskit.circuit import QuantumCircuit
from qiskit_aer import StatevectorSimulator

In [2]:
def encode(bases, states):
    message = []
    rpr = []
    for b,s in zip(bases,states):
        qc = QuantumCircuit(1,1)
        match b,s:
            case 0,0: rpr.append("|0>")
            case 0,1: qc.x(0), rpr.append("|1>")
            case 1,0: qc.h(0),rpr.append("|+>")
            case 1,1: qc.x(0), qc.h(0), rpr.append("|->")
        message.append(qc)
    
    print(f"Encoded msg: \t{''.join(rpr)}")
    return message

def decode(msg: list[QuantumCircuit], bases):
    backend = StatevectorSimulator()
    states = []
    for m,b in zip(msg,bases):
        match m,b:
            case _,0: m.measure(0,0)
            case _,1: m.h(0), m.measure(0,0)
        states.append(int(backend.run(circuits=m, shots=1,memory=True)
                          .result()
                          .get_memory()[0]))
    return states


In [3]:
def env_init(n_bits: int, intercepted=False):
    num_qubits = n_bits

    alice_bases = np.random.randint(2, size=num_qubits)
    alice_bits = np.random.randint(2, size=num_qubits)
    bob_bases = np.random.randint(2, size=num_qubits)

    eve_bases = []
    if intercepted:
        eve_bases= np.random.randint(2, size=num_qubits)
    return alice_bits, alice_bases, bob_bases, eve_bases

In [4]:
def simulate_bb84(n_bits: int, alice_bits, alice_bases, bob_bases, eve_bases=[]):
    """return decoded message containing states in |0> or |1>"""
    msg = encode(alice_bases, alice_bits)
    # print(f"Alice's Encoded Message:\t {np.array2string(alice_encoded_key)}")

    if len(eve_bases):
        eve_eavesdrop = np.array(decode(msg, eve_bases))
        print(f"Eve's message:\t {np.array2string(eve_eavesdrop)}")
    
    #If basis is a Pauli matrix (invert = itself), the measured result will match if the basis is matched
    bob_key = np.array(decode(msg, bob_bases))
    print(f"Bob's message: \t {np.array2string(bob_key)}")

    return bob_key
def sample_bits(bits, selection):
    sample = []
    for i in selection:
        i = np.mod(i, len(bits))
        sample.append(bits[i])
    return sample

def remove_bargage(pubA, pubB, sec):
    good_bits = []
    for i in range(len(pubA)):
        if pubA[i] == pubB[i]:
            good_bits.append(sec[i])
    return good_bits    

In [5]:


nbits = 12

alice_bits, alice_bases,bob_bases,eve_bases = env_init(nbits,intercepted=True)
alice_bits = np.array([1,1,0,0,0,0,1,1,0,0,1,0])
alice_bases = np.array([0,0,1,0,0,0,1,1,1,1,1,0])
bob_bases = np.array([1,0,1,0,1,1,1,0,1,1,1,0])
# eve_bases = [0,1,1,1,1,0,0,0,0,0,1,1]
print(f"Alice's message: {np.array2string(alice_bits)}")
print(f"Alice's basis: \t {' '.join(['X' if b else 'Z' for b in alice_bases])}")
print(f"Bob's basis: \t {' '.join(['X' if b else 'Z' for b in bob_bases])}")

if len(eve_bases):
    print(f"Eve's basis: \t {' '.join(['X' if b else 'Z' for b in eve_bases])}")
    


bob_decoded = simulate_bb84(nbits, alice_bits, alice_bases, bob_bases,eve_bases=eve_bases)

print("Remove bargage...")
alice_key = remove_bargage(alice_bases, bob_bases, alice_bits)
bob_key = remove_bargage(alice_bases, bob_bases, bob_decoded)
print(f"Alice's key: \t{alice_key}")
print(f"Bob's key:  \t{bob_key}")

sample_size = 7
bit_selection = np.random.randint(nbits, size=sample_size)
print(f"Select at\t{bit_selection}")
bob_sample = sample_bits(bob_key, bit_selection)
alice_sample = sample_bits(alice_key, bit_selection)

print(f"Alice's sample:\t{alice_sample}")
print(f"Bob's sample:  \t{bob_sample}")
if bob_sample != alice_sample:
    print("Intercepted!")

Alice's message: [1 1 0 0 0 0 1 1 0 0 1 0]
Alice's basis: 	 Z Z X Z Z Z X X X X X Z
Bob's basis: 	 X Z X Z X X X Z X X X Z
Eve's basis: 	 X X Z Z X X Z X X X X Z
Encoded msg: 	|1>|1>|+>|0>|0>|0>|->|->|+>|+>|->|0>
Eve's message:	 [0 0 1 0 1 0 1 1 0 0 1 0]
Bob's message: 	 [1 0 1 0 0 1 0 1 0 0 0 0]
Remove bargage...
Alice's key: 	[1, 0, 0, 1, 0, 0, 1, 0]
Bob's key:  	[0, 1, 0, 0, 0, 0, 0, 0]
Select at	[11  0  7  9  1 11 11]
Alice's sample:	[1, 1, 0, 0, 0, 1, 1]
Bob's sample:  	[0, 0, 0, 1, 1, 0, 0]
Intercepted!
